### Toy problem 1: Formulate and solve flow optimization toy problem as MINLP and QUBO

Obj Fcn: Minimize the total cost of flow <br> <br>
Constraints: <br>
&nbsp; 1. Flow conservation: The flow into each node equals the flow out of each node <br>
&nbsp; 2. At crossroads, the flow must take one of the two possible paths (binary constraint) <br>


In [23]:
import pyomo.environ as pyo
import numpy as np 

## MINLP Formulation

# Problem data 
flow_network = {'f1': ('s1', 's2'), 'f2': ('s2', 's3'), 
                'f3': ('s3','s4'), 'f4': ('s3', 's5'), 
                'f5': ('s4', 's6'), 'f6': ('s5', 's6'),
                'f7': ('s6', 's7')}

flow_cost = {'f1': 1, 'f2': 1, 'f3': 3, 'f4': 8, 'f5': 4, 'f6': 9, 'f7': 1}

total_flow = 2

# penalty weights for binary constraints
Ky = 10
Kz = 4

# Model
model = pyo.ConcreteModel(doc='Flow Optimization Problem')

# Sets
model.nodes = pyo.Set(initialize=['s1', 's2', 's3', 's4', 's5', 's6', 's7'], doc='nodes') 
model.edges = pyo.Set(initialize=flow_network.keys(), doc='edges')

# Decision Variables  
model.f = pyo.Var(model.edges, domain=pyo.NonNegativeReals, doc='flow on each edge')
model.y = pyo.Var(domain=pyo.Binary, doc='binary decision variable for f3')
model.z = pyo.Var(domain=pyo.Binary, doc='binary decision variable for f4')

# Parameters
model.fcost = pyo.Param(model.edges, initialize=flow_cost, doc='cost of flow on each edge', mutable=True)

# Objective Function (minimize cost)
model.totalcost = pyo.Objective(expr=sum(model.f[e]*model.fcost[e] for e in model.edges) + Ky*model.y + Kz*model.z, sense=pyo.minimize)


# Constraints 

# Disjunction Constraint
def disjunction_rule(model):
    return model.y + model.z == 1
model.disjunction = pyo.Constraint(rule=disjunction_rule, doc='disjunction constraint')


# Flow Conservation Constraints
def flow_conservation_rule(model, node):
    inflow = sum(model.f[edge] for edge in model.edges if flow_network[edge][1] == node)
    outflow = sum(model.f[edge] for edge in model.edges if flow_network[edge][0] == node)
    if node == 's1':
        return outflow == total_flow  # source node
    elif node == 's7':
        return inflow == total_flow  # sink node
    else:
        return inflow == outflow  # intermediate nodes

model.flow_conservation = pyo.Constraint(model.nodes, rule=flow_conservation_rule, doc='flow conservation constraints')

model.xrd1 = pyo.Constraint(expr=model.f['f3'] * model.z <= 0, doc='linking z with f3')
model.xrd2 = pyo.Constraint(expr=model.f['f4'] * model.y <= 0, doc='linking y with f4') 
# model.xrd3 = pyo.Constraint(expr=model.f['s3_s4'] + model.f['s3_s5'] <= model.f['s2_s3'], doc='conservation of flow at s3')

# Solve the model using a solver
solver = pyo.SolverFactory('gurobi',solver_io='python')
results = solver.solve(model, tee=True)

model.pprint()
print(results)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: 13th Gen Intel(R) Core(TM) i7-1365U, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 8 rows, 9 columns and 16 nonzeros
Model fingerprint: 0x1ee5e4a3
Model has 2 quadratic constraints
Variable types: 7 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 7 rows and 7 columns
Presolve time: 0.00s
Presolved: 3 rows, 3 columns, 6 nonzeros
Variable types: 0 continuous, 3 integer (1 binary)
Found heuristic solution: objective 44.0000000

Root relaxation: objective 3.000000e+01, 0 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf 

In [26]:
# Delete constraints and variables
del model.totalcost
# del model.disjunction

# Additional parameter for penalty weights
P = 11

# Moving constraints to the objective 
# disjunction constraint
model.totalcost = pyo.Objective(expr=sum(model.f[e]*model.fcost[e] for e in model.edges) + Ky*model.y + Kz*model.z + P*(1 - (model.y + model.z)), sense=pyo.minimize)


# Solve the model using a solver
solver = pyo.SolverFactory('gurobi',solver_io='python')
results = solver.solve(model, tee=True)

model.pprint()
print(results)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: 13th Gen Intel(R) Core(TM) i7-1365U, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 7 rows, 9 columns and 14 nonzeros
Model fingerprint: 0xfff95e0e
Model has 2 quadratic constraints
Variable types: 7 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve removed 6 rows and 5 columns
Presolve time: 0.00s
Presolved: 5 rows, 6 columns, 10 nonzeros
Variable types: 2 continuous, 4 integer (2 binary)
Found heuristic solution: objective 30.0000000

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 12 (of 12 available processors)

Notes: <br>
The optimal objective function value and f values are the same; however, the binary variables y and z are both set to 0. <br>
 - The objective function value is the same because the penalty weights (P) and (Ky), for disjunction violation and y=1 respectively, are the same value at 10. 

How to enforce the disjunction when the constraint is moved to the objective? 
 - increase the penalty weight (P) to larger than (Ky).

In [29]:
# Delete flow conservation constraints
del model.flow_conservation 

In [32]:
# Delete constraints and variables
del model.totalcost

# disjunct penalty 
Pd = 15  

# Moving constraints to the objective 
# disjunction constraint and flow conservation constraints
model.totalcost = pyo.Objective(expr=sum(model.f[e]*model.fcost[e] for e in model.edges) + Ky*model.y + Kz*model.z 
                                + Pd*(1 - (model.y + model.z)) 
                                + P*((model.f['f2'] - (model.f['f3'] + model.f['f4'])) 
                                     + ((model.f['f5'] + model.f['f6']) - model.f['f7'])
                                     + (model.f['f1'] - model.f['f2']) 
                                     + (model.f['f3'] - model.f['f5']) 
                                     + (model.f['f4'] - model.f['f6']) 
                                     + (total_flow - model.f['f7'])
                                     + (total_flow - model.f['f1'])), sense=pyo.minimize)


# Solve the model using a solver
solver = pyo.SolverFactory('gurobi',solver_io='python')
results = solver.solve(model, tee=True)

model.pprint()
print(results)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: 13th Gen Intel(R) Core(TM) i7-1365U, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 0 rows, 9 columns and 0 nonzeros
Model fingerprint: 0x2501090d
Model has 2 quadratic constraints
Variable types: 7 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 59.0000000
Found heuristic solution: objective 43.0000000
Presolve removed 0 rows and 4 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 2: 43 59 

Model is unbounded
Best objective 4.300000000000e+01, best bound -, gap -
model.name="unkno

## Simplified Flow Network 
Leveraging some of the equalities in the problem, the flow network can be simplified <br>
> 7 --> 4 nodes (f1 = f2; f3 = f5; f4 = f6) <br>
> 7 --> 4 edges (f1 = f2; f3 = f5; f4 = f6)

Additionally, the edge can be written as binary variables to denote flow or no flow on that edge, and the total_flow can be used as a weight to binary variable: 
> total flow on edge i = total_flow * fi where fi = 1 if flows through the edge, fi = 0 if no flow.   

In [16]:
import pyomo.environ as pyo
import numpy as np 

## MINLP Formulation

# Problem data 
flow_network = {'f1': ('s1', 's2'), 'f2': ('s2', 's3'), 
                'f3': ('s2','s3'), 'f4': ('s3', 's4')
                }

flow_cost = {'f1': 2, 'f2': 7, 'f3': 17, 'f4': 1}

total_flow = 2

# penalty weights for binary constraints
K2 = 10
K3 = 4

# Model
model = pyo.ConcreteModel(doc='Flow Optimization Problem')

# Sets
model.nodes = pyo.Set(initialize=['s1', 's2', 's3', 's4'], doc='nodes') 
model.edges = pyo.Set(initialize=flow_network.keys(), doc='edges')

# Decision Variables  
model.f = pyo.Var(model.edges, domain=pyo.Binary, doc='flow binary decision on each edge')

# Parameters
model.fcost = pyo.Param(model.edges, initialize=flow_cost, doc='cost of flow on each edge', mutable=True)

# Objective Function (minimize cost)
model.totalcost = pyo.Objective(expr=sum(model.f[e]*model.fcost[e] for e in model.edges) * total_flow + K2*model.f['f2'] + K3*model.f['f3'], sense=pyo.minimize)


# Constraints 
# Disjunction Constraint
def disjunction_rule(model):
    return model.f['f2'] + model.f['f3'] == 1
model.disjunction = pyo.Constraint(rule=disjunction_rule, doc='disjunction constraint')

# Flow Conservation Constraints
def flow_conservation_rule(model, node):
    inflow = sum(model.f[edge] for edge in model.edges if flow_network[edge][1] == node)
    outflow = sum(model.f[edge] for edge in model.edges if flow_network[edge][0] == node)
    if node == 's1':
        return outflow * total_flow == total_flow  # source node
    elif node == 's4':
        return inflow * total_flow == total_flow  # sink node
    else:
        return inflow == outflow  # intermediate nodes

model.flow_conservation = pyo.Constraint(model.nodes, rule=flow_conservation_rule, doc='flow conservation constraints')

model.xrd0 = pyo.Constraint(expr=model.f['f2'] + model.f['f3'] == 1, doc='f2 or f3 constraint')

# model.xrd1 = pyo.Constraint(expr=model.f['f2'] + model.f['f3'] <= 1, doc='f2 or f3 constraint')
# model.xrd2 = pyo.Constraint(expr=model.f['f2'] + model.f['f3'] >= 1, doc='f2 or f3 constraint')

# model.xrd1 = pyo.Constraint(expr=model.f['f2'] * model.f['f3'] == 0, doc='f2 or f3 constraint')

# Solve the model using a solver
solver = pyo.SolverFactory('gurobi',solver_io='python')
results = solver.solve(model, tee=True)

model.pprint()
print(results)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: 13th Gen Intel(R) Core(TM) i7-1365U, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 6 rows, 4 columns and 12 nonzeros
Model fingerprint: 0x504aa310
Variable types: 0 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e+00, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 6 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 30 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+01, best bound 3.000000000000e+01, gap 0.0000%
Flow Optimization Problem

    2 Set Declarations
        edges : edges
            Size=1, In